In [1]:
import importlib

import siamese_network
importlib.reload(siamese_network)
from siamese_network import SiameseNet

import siamese_api
importlib.reload(siamese_api)
from siamese_api import SiameseAPI

import data_generator as dg
importlib.reload(dg)

import grouping
importlib.reload(grouping)
from grouping import Grouping

import pandas as pd, random, pickle, numpy as np, tables, sys, time

/libs/base/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sapi = SiameseAPI(model_file_path='data/siamese_model_new.h5', max_words=50, embedding_size=64, batch_size=32)
sapi.get_model()
sapi.model.init_model()
sapi.model.load()
sapi.kdtree = dg.load_data_pkl('kdtree.pkl')

In [26]:
sentence = "Women Casual V-Neck Short Sleeve Solid Elastic Pullover T-Shirt Top ECBY"

In [27]:
start = time.time()
ind = sapi.get_nearest_neighbors(sentence)
print(time.time()-start)

0.41991591453552246


In [9]:
print(sapi.benchmark_kdtree(num_samples=1000))

(0.3964170196056366, 0.39590346813201904, 0.9287691116333008)


In [29]:
times = []
for i in random.sample(range(len(items)), 1000):
    start = time.time()
    ind = tree.query_radius([embeddings[i]], r=0.693)
    times.append(time.time()-start)

In [33]:
np.mean(times)*1000

43.428667068481445

In [19]:
items[1][2]

'Women Casual V-Neck Short Sleeve Solid Elastic Pullover T-Shirt Top ECBY'

In [26]:
for x in ind[0]:
    print(items[x][2])

Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of Ohio State University and OSU Buckeyes Bearcats  Artix Mens Shirts
Hamilton Ohio T-Shirt Home of 

In [10]:
validation_set = pd.read_csv("grouping_validation_set.csv")
gs_items = pd.read_csv("items_gs.csv")

In [11]:
gs_items.columns

Index(['Unnamed: 0', 'item_id', 'title'], dtype='object')

In [12]:
import collections

base_item_ids = list(validation_set['Item Id'])
grouped_item_ids = list(validation_set['Items to be grouped'])

gs_mapping = collections.defaultdict(list)
for i in range(len(base_item_ids)):
    gs_mapping[base_item_ids[i]].append(grouped_item_ids[i])

In [13]:
gs_item_title_map = dict()
gs_item_titles = list(gs_items['title'])
gs_item_ids = list(gs_items['item_id'])

for i in range(len(gs_item_ids)):
    gs_item_title_map[gs_item_ids[i]] = gs_item_titles[i]

In [19]:
sapi = SiameseAPI(model_file_path='data/siamese_model_new.h5', max_words=50, embedding_size=64, batch_size=32)
sapi.get_model()
sapi.model.init_model()
sapi.model.load()

In [33]:
gs_embeddings = sapi.model.get_embeddings(gs_item_titles)

In [40]:
dg.save_data_pkl(gs_embeddings, 'gs_embeddings.pkl')

In [41]:
gs_item_embed_map = dict()

for i in range(len(gs_item_ids)):
    gs_item_embed_map[gs_item_ids[i]] = gs_embeddings[i]

In [42]:
from sklearn.neighbors import KDTree
sapi.kdtree = KDTree(gs_embeddings, leaf_size=50)

In [53]:
num_predicted, num_actual, num_correct = 0, 0, 0

for base_item_id, grouped_item_ids in gs_mapping.items():
    if base_item_id in gs_item_embed_map:
        nearest = sapi.kdtree.query_radius([gs_item_embed_map[base_item_id]], r=0.693)[0]
        predicted_item_ids = [gs_item_ids[x] for x in nearest]
        num_correct += len(set(grouped_item_ids).intersection(set(predicted_item_ids)))
        num_predicted += len(predicted_item_ids)
        num_actual += len(grouped_item_ids)

precision = float(num_correct)/num_predicted
recall = float(num_correct)/num_actual

print(precision)
print(recall)

0.9724247646824871
0.3891040263116015


In [11]:
items = dg.load_data_pkl('items.pkl')

In [3]:
grp_instance = Grouping(items)
groups = grp_instance.true_grouping()

In [ ]:
print "Fetching important attributes..."
var_instance1 = Variant(groups1, items, max_attrs_per_var=1, max_variants=10, excluded_attrs=excluded_attrs)
valid_attrs = set([attr[0] for attr, score in var_instance1.get_variant_scores()])
    
print "Fetching variant criteria..."
var_instance = Variant(groups1, items, max_attrs_per_var=2, max_variants=1, excluded_attrs=excluded_attrs, valid_attrs=valid_attrs)
variant_scores = var_instance.get_variant_scores()

print "Predominant Variant Criteria : ", variant_scores[0]

print "Fetching variant criteria per group..."
pred_grp_variants = var_instance.get_predicted_variants()

print "Results"
var_instance.results(pred_grp_variants, variant_scores[0][0])

In [ ]:
from word2vec_api import Word2VecAPI
wapi = Word2VecAPI(items)
wapi.train_model()
wapi.insert_embeddings()

In [ ]:
import time, random
times = []
for i in random.sample(range(len(items)), 1000):
    start = time.time()
    d = wapi.get_nearest_neighbors_mongo(items[i][2], groups, head_thres=0.5, ind_thres=0.999)
    times.append(time.time()-start)

In [ ]:
import numpy as np
np.mean(times)

In [ ]:
items[0][2]

In [ ]:
for x in d:
    print x

In [ ]:
w_embeds = wapi.get_representations([str(x[2]) for x in items])

In [ ]:
grp_instance2 = Grouping(items, representations=w_embeds, similarity_threshold=0.1)
grp_instance2.init_groups()
groups2 = grp_instance2.auto_groups
print grp_instance2.get_clustering_scores()

In [ ]:
len(groups2)

In [ ]:
print "Fetching important attributes..."
var_instance1 = Variant(groups2, items, max_attrs_per_var=1, max_variants=10, excluded_attrs=excluded_attrs)
valid_attrs = set([attr[0] for attr, score in var_instance1.get_variant_scores()])
    
print "Fetching variant criteria..."
var_instance = Variant(groups2, items, max_attrs_per_var=2, max_variants=1, excluded_attrs=excluded_attrs, valid_attrs=valid_attrs)
variant_scores = var_instance.get_variant_scores()

print "Predominant Variant Criteria : ", variant_scores[0]

print "Fetching variant criteria per group..."
pred_grp_variants = var_instance.get_predicted_variants()

print "Results"
var_instance.results(pred_grp_variants, variant_scores[0][0])

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np
tree1 = KDTree(s_embeds, leaf_size=50)
tree2 = KDTree(w_embeds, leaf_size=50)

In [ ]:
ind = tree1.query_radius(s_embeds[2], r=0.02)

In [ ]:
ind = tree2.query_radius(w_embeds[2], r=0.02)

In [ ]:
for x in ind[0]:
    print items[x][2]

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np
tree = KDTree(np.tile(embeds.T, 50).T, leaf_size=50)

In [ ]:
import pickle
with open('tree.pkl', 'wb') as f:
    pickle.dump(tree, f)

In [ ]:
import time, random

out = []
for j in range(5, 50, 5):
    tree = KDTree(np.tile(embeds.T, j).T, leaf_size=50)
    w = random.sample(range(len(items)), 1000)
    times = []
    for i in w:
        start = time.time()
        ind = tree.query_radius(embeds[i], r=0.02)
        duration = time.time()-start
        times.append(duration)
    out.append((j, np.max(times)))

In [ ]:
out = out + [(50, 0.028570890426635742)]

In [ ]:
import random
w = random.sample(range(len(items)), 1000)
times = []
for i in w:
    start = time.time()
    ind = tree.query_radius(embeds[i], r=0.02)
    duration = time.time()-start
    times.append(duration)

In [ ]:
175*100

In [ ]:
%matplotlib inline
from matplotlib.pyplot import plot

x, y = zip(*out)
plot(x, y)

In [8]:
22280.845237731934/1024

21.75863792747259

In [8]:
import tables
sents_arr_file = tables.open_file('data/sent_arrays.h5', mode='r')
sents_arr = sents_arr_file.root.data

In [2]:
sents_arr[0]

array([169372, 403212, 485034, 320809, 248157, 402627,  42651, 457089,
       287069, 314260, 402629, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922], dtype=int32)

In [15]:
word2idx_map = dg.load_data_pkl('word2idx_map.pkl')

In [27]:
[dg.word_to_idx(w.decode('utf-8'), word2idx_map) for w in sent_tokens[0]]

[169372,
 403212,
 485034,
 320809,
 248157,
 402627,
 42651,
 457089,
 287069,
 314260,
 402629,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922,
 112922]

In [20]:
import numpy as np
sent_arr_1 = [np.asarray([dg.word_to_idx(w, word2idx_map) for w in sent_tokens[0]])]
sent_arr_2 = [np.asarray([dg.word_to_idx(w, word2idx_map) for w in sent_tokens[1]])]

pred = siamese_net.predict([sent_arr_1, sent_arr_2])[0]

NameError: name 'word2idx_map' is not defined

In [9]:
items[500][2]

'Maui Hawaii Boy Youth Shirts T-Shirt Tee'

In [5]:
import tables

def get_items_sampled(items, num_groups=10000):
    try:
        tokens_file = tables.open_file('data/sent_tokens.h5', mode='r')
        sent_tokens = tokens_file.root.data

        grp_instance = Grouping(items)
        groups = grp_instance.true_grouping()

        group_ids = [key for key, vals in groups.items() if len(vals) >= 5]

        random.seed(42)
        random_group_ids = random.sample(group_ids, num_groups)

        selected_items_indices = []
        for gid in random_group_ids:
            selected_items_indices += groups[gid]

        selected_items_indices = sorted(selected_items_indices)
        print(len(selected_items_indices))

        items = [items[i] for i in selected_items_indices]
        sent_tokens_out = sent_tokens[selected_items_indices,:]
        
        tokens_file.close()

        return items, sent_tokens_out
    
    finally:
        tokens_file.close()

In [6]:
items, sent_tokens = get_items_sampled(items, num_groups=1000)

17469


In [9]:
items[500][2]

"Cancer Awareness T-Shirt Kiss Breast Cancer GoodBye Cancer Cancer Awareness Ribbon Cancer Support Gifts Artix Women's Slouchy T-Shirt Clothes"

In [10]:
sent_tokens[500]

array([b'b"cancer', b'awareness', b't-shirt', b'kiss', b'breast',
       b'cancer', b'goodbye', b'cancer', b'cancer', b'awareness',
       b'ribbon', b'cancer', b'support', b'gift', b'artix', b"women's",
       b'slouchy', b't-shirt', b'clothes"', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT',
       b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'PAD_TXT', b'P

In [4]:
import data_generator as dg
data_pairs = dg.load_data_pkl("train_data_pairs.pkl")

In [6]:
import numpy as np
items1, items2, labels = zip(*data_pairs)
items1, items2, labels = np.array(items1), np.array(items2), np.array(labels)

In [17]:
print(items1[0:10].tolist())

[3834845, 355485, 10761044, 816603, 12018381, 7067327, 6523749, 4716772, 3814661, 145964]


In [35]:
sents_arr[[1,1],:]

IndexError: Selection lists cannot have repeated values

In [34]:
sents_arr[1]

array([224237, 257937, 503506, 458512, 462271, 465599, 296584, 430360,
       477761, 488932, 294699, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922, 112922, 112922, 112922, 112922, 112922, 112922,
       112922, 112922], dtype=int32)

In [27]:
import math
g = lambda x: math.exp(-x)
g(0.2)

0.8187307530779818

In [32]:
g(0.5)

0.4965853037914095